In [15]:
import requests
import xml.etree.ElementTree as ET

s = requests.Session() # create session
# Post login credentials to session:
s.post('https://websvc.biocyc.org/credentials/login/', data={'email':'dalba@uw.edu', 'password':'RNAdevices2024'})


In [195]:
# from EC to BioCyc Reaction ID
EC = '1.1.1.37'
r = s.get('https://metacyc.org/META/substring-search?type=NIL&object=EC+'+EC)

In [196]:
loc = r.text.find('/META/NEW-IMAGE?type=REACTION&object=') # this only finds the first instance, there may be other reactinos associated with this EC...
BioCycID = r.text[loc+37:].split('"')[0]

In [198]:

# from BioCyc Reaction ID to  reactions
r = s.get('https://websvc.biocyc.org/apixml?fn=enzymes-of-reaction&id=META:{i}&detail=low'.format(i=BioCycID))
reactions = [e.find('catalyzes').find('Enzymatic-Reaction').items()[0][1] for e in list(ET.fromstring(r.text))[1:]]

# from reactions to inhibitors
inhibitors = []
for reaction in reactions:
    r = s.get('https://websvc.biocyc.org/apixml?fn=direct-inhibitors&id={i}&detail=low'.format(i=reaction))
    inhibitors.append([list(c)[-1].text for c in list(ET.fromstring(r.text))[1:]])
inhibitors = sum(inhibitors,[])

In [204]:
inhibitors

['oxaloacetate',
 'ATP',
 'acetyl-CoA',
 'coenzyme A',
 'oxalate',
 '<i>N</i>-ethylmaleimide',
 "5,5'-dithio-bis-2-nitrobenzoate",
 "4,4'-dithiodipyridine",
 'diethylpyrocarbonate',
 'diacetyl',
 'Cd<SUP>2+</SUP>']